In [1]:
import torch 
import torch.nn as nn 

from llama2 import Llama2Model

In [ ]:
LLAMA2_CONFIG_7B = {
    "vocab_size": 32000,     # Vocabulary size
    "ctx_len": 4096,         # Context length
    "d_model": 4096,         # Embedding dimension
    "n_heads": 32,           # Number of attention heads
    "n_layers": 32,          # Number of layers
    "d_ff": 11008,           # NEW: Size of the intermediate dimension in FeedForward
    "dtype": torch.bfloat16  # NEW: Lower-precision dtype to reduce memory usage
}

model = Llama2Model(LLAMA2_CONFIG_7B)

In [ ]:
encoded = [0, 1, 2, 3, 4]
encoded_tensor = torch.tensor(encoded).unsqueeze(0)
encoded_tensor.shape

torch.Size([1, 5])